In [1]:
# Initialize the workspace
from pymongo import MongoClient
import pprint
from bs4 import BeautifulSoup
import requests
import pandas as pd
import time

In [2]:
# Connect with Mongo to place data into a database
client = MongoClient('localhost', 27017)
db = client['metacritic']
test = db['test']

In [4]:
# Test databases/collection
test.insert_one({'abc': 123})

In [5]:
# Create Mongo collections in which to place data
altcountry = db['altcountry']
alternative = db['alternative']
blues = db['blues']
comedy = db['comedy']
country = db['country']
dance = db['dance']
electronic = db['electronic']
experimental = db['experimental']
folk = db['folk']
house = db['house']
indie = db['indie']
jazz = db['jazz']
latin = db['latin']
metal = db['metal']
pop = db['pop']
psychedelic = db['psychedelic']
punk = db['punk']
rap = db['rap']
rb = db['rb']
reggae = db['reggae']
rock = db['rock']
singersongwriter = db['singersongwriter']
soul = db['soul']
soundtrack = db['soundtrack']
techno = db['techno']
vocal = db['vocal']
world = db['world']
coll_list = [altcountry, alternative, blues, comedy, country,
             dance, electronic, experimental, folk, house, indie,
             jazz, latin, metal, pop, psychedelic, punk, rap,
             rb, reggae, rock, singersongwriter, soul,
             soundtrack, techno, vocal, world]

In [11]:
# Create a dictionary of genres and their associated page numbers
genre_dict = {'alt-country': 6, 'alternative': 38, 'blues':6, 'comedy':0,
              'country': 25, 'dance':6, 'electronic': 79, 'experimental':10,
              'folk': 22, 'house':5, 'indie': 74, 'jazz': 13, 'latin':2,
              'metal':6, 'pop':32, 'psychedelic':0, 'punk':4, 'rap':48,
              'rb':26, 'reggae':3, 'rock': 132, 'singer-songwriter':9,
              'soul':7, 'soundtrack':2, 'techno':6, 'vocal':4, 'world':2}

In [12]:
# Define Beautiful Soup parameters
url = 'https://www.metacritic.com/browse/albums/genre/date/{}?page={}'
user_agent = {'User-agent': 'Mozilla/5.0'}

In [13]:
# Loop through all pages in all genre, placing them in MongoDB along the way
for (genre, pages), coll in zip(genre_dict.items(), coll_list):
    for page in range(pages):
        r = requests.get(url.format(genre, page), headers = user_agent)
        soup = BeautifulSoup(r.text, 'html.parser')
        for one_tag in soup.find_all('div', class_ = 'product_wrap'):
            # Album title
            album = one_tag.find('a').text.strip()
            # Artist
            artist = one_tag.find_all('span', class_ = 'data')[0].text.strip()
            # Date
            date = ' '.join(one_tag.find_all('span', class_ = 'data')[1].text.split())
            # Score
            score_ = one_tag.find('div', class_ = "basic_stat product_score brief_metascore").text
            score = int(''.join(char for char in score_ if char.isdigit()))
            # Used this as a way to track potential problems/troubleshoot
            print(genre, album, artist, date, score)
            coll.insert_one({'album':album, 'artist':artist, 
                             'date':date, 'score':score})
        time.sleep(15)

Closer Than Together The Avett Brothers Oct 4 56
III The Lumineers Sep 13 72
The Saint of Lost Causes Justin Townes Earle May 24 74
Psychedelic Country Soul The Long Ryders Feb 15 79
What It Is Hayes Carll Feb 15 79
Dying Star Ruston Kelly Sep 7 73
Old Rockhounds Never Die Odetta Hartman Aug 17 81
Wide Awake Rayland Baxter Jul 13 83
Dove Belly May 4 72
Can't Wake Up Shakey Graves May 4 82
Other Arrangements Parker Millsap May 4 79
Find a Light Blackberry Smoke Apr 6 72
Forever and Then Some Lillie Mae Apr 14 80
Adios Cory Branan Apr 7 70
Graveyard Whistling Old 97's Feb 24 79
Prisoner Ryan Adams Feb 17 79
Notes of Blue Son Volt Feb 17 77
Sweet Creep Jonny Fritz Oct 14 79
Eastside Bulldog Todd Snider Oct 7 75
Redemption & Ruin The Devil Makes Three Sep 16 74
Autumn Chatham County Line Sep 2 77
Exodus of Venus Elizabeth Cook Jun 17 86
Juarez [Reissue] Terry Allen May 20 84
Full Circle Loretta Lynn Mar 4 84
Going Down in History Waco Brothers Feb 26 80
Above the Prairie The Pines Jan 22 8

Big Echo The Morning Benders Mar 9 72
Kairos White Hinterland Mar 9 76
El Turista Josh Rouse Mar 9 75
The Bundles The Bundles Mar 9 58
The Creatures In The Garden Of Lady Walton Clogs Mar 2 75
The Law Of Large Numbers Emma Pollock Mar 2 72
Fallen House Sunken City B. Dolan Mar 2 81
Double Jointer Golden Triangle Mar 2 69
Fire Like This Blood Red Shoes Mar 1 70
Who Killed Sgt. Pepper? The Brian Jonestown Massacre Feb 23 61
Dear God, I Hate Myself Xiu Xiu Feb 23 74
This Addiction Alkaline Trio Feb 23 70
Have One On Me Joanna Newsom Feb 23 85
Work Shout Out Louds Feb 23 68
I Was Trying To Describe You To Someone Crime In Stereo Feb 23 89
Become Secret Picastro Feb 23 81
Pick Of The Litter 1980-2010 Was (Not Was) Feb 23 82
Falling Down A Mountain Tindersticks Feb 16 76
Life Is Sweet! Nice To Meet You Lightspeed Champion Feb 16 72
Minor Love Adam Green Feb 16 70
Field Music (Measure) Field Music Feb 16 79
Peace And Love Juliana Hatfield Feb 16 60
The Constant Story Of The Year Feb 16 65
Pre

Fiction Family Fiction Family Jan 20 50
Haymaker! The Gourds Jan 6 78
Folie A Deux Fall Out Boy Dec 16 73
When The World Comes Down The All-American Rejects Dec 16 64
A Snowflake Fell (And It Felt Like A Kiss) Glasvegas Dec 1 77
Prospekt's March [EP] Coldplay Nov 25 67
Happy In Galoshes Scott Weiland Nov 25 59
Alone II: The Home Recordings Rivers Cuomo Nov 25 71
Dark Horse Nickelback Nov 18 49
The BBC Sessions Belle and Sebastian Nov 18 81
Sycamore Meadows Butch Walker Nov 11 81
Jonas Reinhardt Jonas Reinhardt Nov 11 69
Ode To J. Smith Travis Nov 4 66
Split Thursday/Envy Nov 4 85
Season Of Poison Shiny Toy Guns Nov 4 59
Heart On Eagles of Death Metal Oct 28 74
4:13 Dream The Cure Oct 28 69
Cardinology Ryan Adams Oct 28 72
On Your Sleeve Jesse Malin Oct 28 61
Damn Right Rebel Proud Hank Williams III Oct 21 71
Perfect Symmetry Keane Oct 14 64
Little Honey Lucinda Williams Oct 14 72
Gossip In The Grain Ray LaMontagne Oct 14 73
Secret Machines Secret Machines Oct 14 65
Peace Queer Todd Sni

City Beach Jill Cunniff Feb 20 52
Infinity On High Fall Out Boy Feb 6 75
The Good, The Bad & The Queen The Good, the Bad & the Queen Jan 23 76
Learn To Sing Like A Star Kristin Hersh Jan 23 74
Light Grenades Incubus Nov 28 65
The Devil And God Are Raging Inside Me Brand New Nov 21 78
When Your Heart Stops Beating (+44) Nov 14 60
The Peel Sessions 1991-2004 PJ Harvey Nov 7 77
Saturday Night Wrist Deftones Oct 31 72
In The Absence Of Truth Isis Oct 31 70
The Black Parade My Chemical Romance Oct 24 79
Threes Sparta Oct 24 60
Smoke & Mirrors The Datsuns Oct 9 62
The Information Beck Oct 3 73
Sam's Town The Killers Oct 3 64
The Open Door Evanescence Oct 3 61
Ole! Tarantula Robyn Hitchcock Oct 3 80
The Lemonheads The Lemonheads Sep 26 70
Super Extra Gravity The Cardigans Sep 19 64
Amputechture The Mars Volta Sep 12 61
Welcome To The Drama Club Everclear Sep 12 54
Blood Mountain Mastodon Sep 12 82
Revelations Audioslave Sep 5 60
Riot City Blues Primal Scream Aug 22 59
Razorlight Razorlight Au

Come Feel Me Tremble Paul Westerberg Oct 21 78
Streetcore Joe Strummer & The Mescaleros Oct 21 85
Coral Fang The Distillers Oct 14 71
12 Memories Travis Oct 14 64
Bazooka Tooth Aesop Rock Sep 23 74
The Long Road Nickelback Sep 23 62
Results May Vary Limp Bizkit Sep 23 33
In Reverie Saves the Day Sep 16 77
Ima Robot Ima Robot Sep 16 65
Reality David Bowie Sep 16 74
Lost in Translation OST Original Soundtrack Sep 9 84
Amazing Grace Spiritualized Sep 9 68
You Gotta Go There To Come Back Stereophonics Sep 9 60
The Other Side Of Daybreak Beth Orton Sep 2 66
Amateur Night In The Big Top Shaun William Ryder Sep 2 51
Bazooka!!! The Star Spangles Aug 19 68
Indestructible Rancid Aug 19 84
Welcome To The Monkey House The Dandy Warhols Aug 19 73
truANT Alien Ant Farm Aug 19 63
Everyone Deserves Music Michael Franti & Spearhead Aug 19 71
A Mark, A Mission, A Brand, A Scar Dashboard Confessional Aug 12 77
Get The Picture Smash Mouth Aug 5 61
Killing Joke [2003] Killing Joke Aug 5 79
Quebec Ween Aug 

...

The Harsh Light of Day Fastball Sep 19 64
Music Madonna Sep 19 80
SelmaSongs: Music from the Motion Picture Dancer in th... Björk Sep 19 76
Maroon Barenaked Ladies Sep 12 59
Kelly's Locker [EP] Sarah Cracknell Sep 12 60
Righteous Love Joan Osborne Sep 12 59
Freaked Out and Small The Presidents Of The United States Of America Sep 12 77
It's Like This Rickie Lee Jones Sep 12 65
Red Dirt Girl Emmylou Harris Sep 12 73
OST Almost Famous Sep 12 70
Lift Your Skinny Fists Like Antennas to Heaven Godspeed You! Black Emperor Sep 12 84
Twisted Tenderness Electronic Sep 12 56
Relationship Of Command At the Drive-In Sep 12 77
Secret South 16 Horsepower Sep 12 82
The Januaries The Januaries Sep 12 76
The Good, the Bad, and the Funky Tom Tom Club Sep 12 71
Spectrum of Infinite Scale Man... Or Astroman? Sep 5 65
Private Suit Bettie Serveert Sep 5 80
Hyacinths and Thistles The 6ths Sep 5 68
Excuses for Travelers Mojave 3 Sep 5 77
With Ghost Damon & Naomi Sep 5 80
Early Days Beth Hirsch Aug 29 51
The Co

Unfamiliar Faces Matt Costa Jan 22 59
Theology Sinéad O'Connor Jun 26 50
The Brag And Cuss Rocky Votolato Jun 19 78
Sweet Warrior Richard Thompson May 29 84
Release The Stars Rufus Wainwright May 15 72
American Doll Posse Tori Amos May 1 69
The Reminder Feist May 1 79
Strangelet Grant-Lee Phillips Mar 27 76
Saltbreakers Laura Veirs Mar 27 78
Life In Cartoon Motion Mika Mar 27 55
Glitter In The Gutter Jesse Malin Mar 20 67
If The Ocean Gets Rough Willy Mason Mar 20 66
These Friends Of Mine Rosie Thomas Mar 13 67
My Name Is Buddy Ry Cooder Mar 6 76
Dressed Up For The Letdown Richard Swift Feb 20 78
Ash Wednesday Elvis Perkins Feb 20 77
West Lucinda Williams Feb 13 69
Like, Love, Lust & The Open Halls Of The Soul Jesse Sykes Feb 6 77
Children Running Through Patty Griffin Feb 6 87
The Sermon On Exposition Boulevard Rickie Lee Jones Feb 6 75
Woke Myself Up Julie Doiron Jan 23 75
Learn To Sing Like A Star Kristin Hersh Jan 23 74
9 Damien Rice Nov 14 64
Pretty Little Head Nellie McKay Oct 31

My True Story Aaron Neville Jan 22 75
No Beginning No End José James Jan 22 79
American Soul Mick Hucknall Nov 6 60
A Dream Deferred Skyzoo Oct 9 85
Thankful N' Thoughtful Bettye LaVette Sep 25 79
The  Glorious Dead The Heavy Aug 21 74
Big Inner Matthew E. White Aug 21 85
The Soul Sessions, Vol. 2 Joss Stone Jul 31 60
One Day I'm Going To Soar Dexys Jun 4 82
Heaven Rebecca Ferguson May 29 84
Night & Day Andre Williams & The Sadies May 15 75
What the World Needs Now The Sugarman 3 May 14 83
Time's All Gone Nick Waterhouse Apr 30 71
Tuskegee Lionel Richie Mar 27 74
Covered Macy Gray Mar 27 68
Faithful Man Lee Fields Mar 13 77
Ice Level Ava Luna Mar 6 68
Hoods and Shades Andre Williams Feb 28 58
Listen, Whitey! The Sounds of Black Power 1967-1974 Various Artists Feb 28 82
The Promise Mitch Ryder Feb 13 67
Let It Burn Ruthie Foster Jan 31 76
Soul 2 Seal Jan 24 70
Betty Wright: The Movie Betty Wright Nov 15 75
Come Through for You Javier Colon Nov 15 57
The  Dreamer Etta James Nov 8 78
Soul

Loud Patterns Makeness Apr 6 75
Dead Start Program John Tejada Feb 9 73
Impossible Star Meat Beat Manifesto Jan 19 72
Joli Mai Daphni Nov 3 78
Mnestic Pressure Lee Gamble Oct 20 80
World of the Waking State Steffi Sep 22 84
Bioprodukt Ekoplekz Jun 16 78
Versus Carl Craig May 5 78
Abysma Geotic Mar 31 63
Samurai Joakim Mar 24 69
No Future Moire Feb 17 80
An Act of Love Earthen Sea Feb 17 79
DJ-Kicks Matthew Dear Jan 27 82
Fabric 91 Nina Kraviz Dec 9 81
W:/2016ALBUM/ Deadmau5 Dec 2 63
Broken Knowz Jay Daniel Nov 25 80
No Further Ahead Than Today Ulrich Schnauss Nov 4 69
And Michael Mayer Oct 28 76
Joy Brandt Brauer Frick Oct 28 61
Box Gas Oct 28 93
Physicalist Forma Sep 23 79
Psi patten Sep 16 79
The Disco's of Imhotep Hieroglyphic Being Aug 5 80
Night Melody Rival Consoles Aug 5 75
Hangin' at the Beach Delroy Edwards Aug 5 74
DJ-Kicks Jackmaster Jul 8 78
LateNightTales Ólafur Arnalds Jun 24 72
Victorious Floorplan Jun 24 77
Alpha Audion Jun 10 71
The Digging Remedy Plaid Jun 10 71
Good 

Nigeria 70: No Wahala (Highlife, Afro-Funk & Juju 1973... Various Artists Mar 29 82
One People One World Femi Kuti Feb 23 71
Live At Carnegie Hall Caetano Veloso/David Byrne Mar 13 70
San Patricio The Chieftains Featuring Ry Cooder Mar 9 78
Ali & Toumani Ali Farka Toure And Toumani Diabate Feb 23 89
The Best Of The Black President [Reissue] Fela Kuti Oct 26 93
Light A Candle Forro In The Dark Oct 13 66
Warm Heart Of Africa The Very Best Oct 6 77
Fool's Gold Fool's Gold Sep 29 76
Haih...Or Amortecedor Os Mutantes Sep 8 74
Television Baaba Maal Aug 25 73
Imidiwan: Companions Tinariwen Jun 29 84
Espoi Hermas Zopoula May 19 61
Welcome To Mali Amadou & Mariam Mar 24 86
Thank You Very Quickly Extra Golden Mar 10 76
Day By Day Femi Kuti Nov 18 73
In A Town Called Addis Dub Colossus Oct 7 79
In The 7th Moon, The Chief Turned Into A Swimming Fish... Kasai Allstars Jul 15 83
Hera Ma Nono Extra Golden Oct 9 75
La Radiolina Manu Chao Sep 4 76
5:55 Charlotte Gainsbourg Apr 24 72
Security Antibalas 